In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 345370.84266243,  342665.99894833],
        [ 345746.23251247,  346284.28258555],
        [ 620596.84288011,  619127.94066601],
        [ 665444.47618312,  661904.52062703],
        [ 176851.19601619,  171276.34189061],
        [ 207120.52999045,  201014.94823914],
        [ 396369.88731884,  402439.12627338],
        [ 405335.12962741,  418074.06936383],
        [ 338359.20343074,  342978.63808599],
        [ 395401.15335005,  391125.24783867],
        [ 428983.30569498,  455566.29955729],
        [ 353127.7849623 ,  358448.11355973],
        [  94383.74259727,   89424.46592153],
        [ 101478.49007601,   98943.95641294],
        [ 165000.49323792,  144858.83413119],
        [ 151595.76112167,  152648.03137783],
        [ 205159.74328156,  210620.20309418],
        [ 215668.19526627,  217724.83974359],
        [ 366035.72042202,  354082.9977506 ],
        [ 275774.75903642,  282746.49135658],
        [ 540065.41748488,  534162.16405245],
        [ 566368.56227152,  544909

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e-02, 4e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.52701788e+07  0.00000000e+00  5.02e+03 1.91e+03  1.00e+06     0s
   1  -2.56884128e+07 -1.17475445e+07  1.17e+03 4.43e+02  2.60e+05     0s
   2  -2.06785548e+07 -2.09658239e+07  8.62e+01 3.28e+01  4.60e+04     0s
   3  -2.05634383e+07 -2.08656944e+07  8.62e-05 3.28e-05  5.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.3501946156382637e-08,
 180476.18675612108,
 64254.99597358357,
 0.0012592570161945038,
 77296.85901033189,
 182401.06071690156,
 0.0011905879380243456,
 3.519779357612034e-07,
 8.360980464028271e-10,
 232069.82826963285,
 8.360973528494176e-10,
 8.360980445159024e-10,
 8.360980468058295e-10,
 8.41733582546234e-10,
 1.4925602746847986,
 1.215347235446662e-08,
 0.001254455847608582,
 0.001265476730953008,
 155426.26131111223,
 6.503106519875276e-05,
 0.0011862521717742602,
 176820.16289296816,
 261762.7058412137,
 45260.36517287859,
 0.0012348833238784995,
 0.001427456707447483,
 973.5049621439566,
 184066.8898988616,
 499523.3212819144,
 1.2153472227884746e-08,
 47959.05641355578,
 60586.140087371445,
 57420.830714460055,
 4.134395678696498e-07,
 107537.84373047954,
 283747.26937405486,
 1.2153472342461199e-08,
 9.780014352536379e-05,
 31115.155008493857,
 93227.70245989485,
 480498.93670387624,
 0.00107699897065483,
 0.00021152373136228836,
 1.2152774433216341e-08,
 1.08479560104839